In [1]:
import pandas as pd
import spacy
import numpy as np
import nltk
nltk.download('stopwords')
#spacy doesnt work properly in collab, doesn't recognize nlp() function

def list_to_phrase(l):
    temp = ''
    for elm in l:
        temp += elm
        temp += ' '
    return temp[:-1]

def remove_all_blanks(l):
    while '' in l:
        l.remove('')

def get_bigrams(l):
    return list(zip(l[:-1],l[1:]))

def prob_given_parent(bigram, corpus): #calculates probability of child given parent in bigram
    corpus = flatten(corpus)
    numerator = corpus.count(bigram)
    zipped_corpus = list(zip(*corpus))
    denom = zipped_corpus[1].count(bigram[1])
    return numerator/denom

def prob_in_corpus(bigram, corpus): #calculates frequency of bigram in corpus
    corpus = flatten(corpus)
    return corpus.count(bigram)/len(corpus)

def child_given_position(pos, child, corpus):
    count = 0
    total = 0
    for review in corpus:
        if len(review) > 1:
            children = list(list(zip(*review))[1])
        elif len(review) == 1: 
            children = [review[0][1]]
        else: 
            children = []
        if child in children:
            total+= 1
        else:
            continue
        if children.index(child) == pos:
            count += 1
    return np.log(count / total)

def create_dict_for_child_given_position(corpus, max_len = 1E99): #corpus should be of [[(),..],...] form
    
    length_list = np.vectorize(len)(corpus)
    max_length = max(length_list)
    if max_length > max_len:
        max_length = max_len
    
    flattened_bigrams = [bigram for elm in corpus for bigram in elm] #this part can be taken out and passed as an argument
    children = list(list(zip(*flattened_bigrams))[1])
    unique_children = list(set(children))
    
    temp = dict()
    
    for child in unique_children:
        for i in range(max_length):
            temp[(i,child)] = child_given_position(i,child, corpus)
    return temp

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scarecrow/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url = "https://raw.githubusercontent.com/CCrisanti/nlp/master/alexa?token=ACNCXEMSWJQ7VREN2WRZFY2562GIY"
reviews = pd.read_csv(url, sep="\n", header=None)
reviews.replace({'‚Äô': "'"}, regex=True)
reviews.replace({'‚äô': "'"}, regex=True)
reviews.replace({'‚Äò': '"'}, regex=True)
reviews.replace({'‚äô': "'"}, regex=True)
print(reviews.head())
stop_words = list(nltk.corpus.stopwords.words('english'))
nlp = spacy.load('en')
#print(reviews.shape)

#creating chunks
reviews = list(reviews.iloc[:,0])




parsed_reviews = [list(nlp(sent).noun_chunks) for sent in reviews]
parsed_reviews = [[str(chunk).split(' ') for chunk in elm] for elm in parsed_reviews]


#removing common pronouns from chunks
for i in range(len(parsed_reviews)):
    for j in range(len(parsed_reviews[i])):
        parsed_reviews[i][j] = [word.lower() for word in parsed_reviews[i][j]]
        for word in stop_words:
            if word.lower() in parsed_reviews[i][j]:
                parsed_reviews[i][j].remove(word)
        parsed_reviews[i][j] =  list_to_phrase(parsed_reviews[i][j]) 
    if '' in parsed_reviews[i]:
        remove_all_blanks(parsed_reviews[i])

#creating bigrams
parsed_reviews_bi = []
for i in range(len(parsed_reviews)):
    parsed_reviews_bi.append(get_bigrams(parsed_reviews[i]))
    
print(parsed_reviews_bi)

                                                   0
0  Great Gift for anyone. Very easy to setup. Coe...
1  Super excited to give this as a gift. It's sup...
2  We bought this for mother in law, buying anoth...
3  Well designed, good sound, has everything Alex...
4  I have one Alexa and three Echo dots and havin...
[[('great gift', 'anyone'), ('anyone', 'iot devices')], [('gift', 'best buy'), ('best buy', 'echo products'), ('echo products', 'store'), ('store', 'amazon')], [('mother', 'law')], [('designed, good sound', 'everything'), ('everything', 'hd video'), ('hd video', 'answers'), ('answers', 'associated video'), ('associated video', 'text'), ('text', 'movie trailers'), ('movie trailers', 'amazon video'), ('amazon video', 'demand'), ('demand', 'amazon compatible cameras'), ('amazon compatible cameras', 'voice'), ('voice', 'message and/or video calls'), ('message and/or video calls', 'amazon show owners'), ('amazon show owners', 'alexa app holders')], [('three echo dots', 'echo sho

In [3]:


import collections
newlist = [item for items in parsed_reviews_bi for item in items]
#print(newlist)
parents = []
children = []
for pair in newlist:
    parents.append(pair[0])
    children.append(pair[1])

parents_counter = collections.Counter(parents)
children_counter = collections.Counter(children)
pair_counter = collections.Counter(newlist)


#returns p(child|parent)
def freq(parent, child): 
    parent_count = parents_counter.get(parent)
    #child_count = children_counter.get(child)
    pair_count = pair_counter.get((parent,child))

    return np.log(pair_count/parent_count)

#print(freq('music', 'lyrics'))


dict_for_freq = {(parent, child): freq(parent, child) for parent, child in newlist}
dict_for_pos = create_dict_for_child_given_position(parsed_reviews_bi, max_len = 3)

print(parents_counter)


/home/scarecrow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log


Counter({'echo show': 118, 'music': 110, 'alexa': 97, 'screen': 93, 'show': 90, 'echo': 73, 'kitchen': 45, 'recipes': 42, 'family': 42, 'device': 36, 'amazon': 36, 'product': 32, 'things': 32, 'videos': 31, 'house': 31, 'gift': 27, 'sound': 27, 'time': 27, 'lot': 26, 'everything': 25, 'dot': 25, 'weather': 25, 'wife': 25, 'video': 25, 'lyrics': 23, 'questions': 23, 'home': 22, 'great product': 20, 'news': 20, 'great addition': 19, 'something': 19, 'one': 18, 'youtube': 18, 'amazon music': 17, 'us': 17, 'voice': 16, 'songs': 16, 'i‚äôm': 16, 'camera': 16, 'echo dot': 16, 'christmas': 16, 'kids': 16, 'thing': 16, 'amazon show': 15, 'phone': 15, 'amazon echo show': 14, 'it‚äôs': 14, 'great device': 13, 'item': 13, 'security cameras': 12, 'speakers': 11, 'son': 11, 'amazon echo': 11, 'times': 11, 'fun': 11, 'video chat': 11, 'lights': 11, 'dots': 11, 'words': 11, 'work': 11, 'purchase': 11, 'touch': 11, 'everyone': 10, 'features': 10, 'love': 10, 'alexa app': 10, 'video calls': 10, 'abilit

In [4]:
print(dict_for_pos)

{(0, 'san diego zoo elephants'): -inf, (1, 'san diego zoo elephants'): -inf, (2, 'san diego zoo elephants'): -inf, (0, 'calendars'): -inf, (1, 'calendars'): -inf, (2, 'calendars'): -inf, (0, 'android, samsung, fire os, ios device'): -inf, (1, 'android, samsung, fire os, ios device'): -inf, (2, 'android, samsung, fire os, ios device'): -inf, (0, '$100 discount'): -inf, (1, '$100 discount'): -inf, (2, '$100 discount'): -inf, (0, 'years'): 0.0, (1, 'years'): -inf, (2, 'years'): -inf, (0, 'numerous security cameras'): -inf, (1, 'numerous security cameras'): -inf, (2, 'numerous security cameras'): -inf, (0, 'video chat'): -2.4849066497880004, (1, 'video chat'): -0.6931471805599453, (2, 'video chat'): -2.4849066497880004, (0, 'commands'): -2.1972245773362196, (1, 'commands'): -2.1972245773362196, (2, 'commands'): -inf, (0, 'password'): -inf, (1, 'password'): -inf, (2, 'password'): 0.0, (0, 'must'): -inf, (1, 'must'): 0.0, (2, 'must'): -inf, (0, 'amazon time'): 0.0, (1, 'amazon time'): -inf, 

In [5]:
#print(reviews)
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/scarecrow/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
reviews_df = pd.read_csv(url, sep="\n", header=None)

for index, row in reviews_df.iterrows():
  print(row[0])
  polarities = sid.polarity_scores(row[0])
  print(polarities)
  print("\n")
#For now all the polarities are just printed but we could easily store them in a list or a dictionary or whatever

Great Gift for anyone. Very easy to setup. Coexist with all IOT Devices. Alexa is AWESOME!
{'neg': 0.0, 'neu': 0.439, 'pos': 0.561, 'compound': 0.9461}


Super excited to give this as a gift. It's super convenient that Best Buy has Echo products in store instead of having to purchase from Amazon.
{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound': 0.9584}


We bought this for mother in law, buying another for me.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


Well designed, good sound, has everything Alexa has plus the HD video. Always ready with answers with associated video or text if applicable. Can show movie trailers, can also watch Amazon video. Excellent on-demand security video with Amazon compatible cameras. Voice activated message and/or video calls to Amazon Show owners or Alexa App holders. Highly recommended.
{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'compound': 0.9501}


I have one Alexa and three Echo dots and having Echo Show now is awesome!
{'neg': 0.0, 'neu': 0.7

{'neg': 0.021, 'neu': 0.797, 'pos': 0.182, 'compound': 0.9955}


Love the echo show so much! Easy to use, connects to your mobil phone to easily call people!
{'neg': 0.0, 'neu': 0.591, 'pos': 0.409, 'compound': 0.885}


9.5/10 Rating ! This has to be one of the best AI products I have used in a while. The screen for an smart home device can be very useful. It really can't compete with the Google home because the Show just brings convenience without having to call it. I primarily use the show to turn on/off my smart lights, simple cooking measurements, simple math, travel time, store location hours, calling, and video calling.... and more. Everything else like, news, calendars, weather are already displayed on screen. Definitely worth it if you plan on using all the minimum features above.
{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'compound': 0.9609}


Love the many features ! Highly recommend it! using the echo saves me lots of unnecessary work... and that‚Äôs amazing!
{'neg': 0.0, 'neu



The Echo Show device connects with a range of services, including Amazon Music to expand your experience. Ask questions and see the answers with the Echo Show, which connects to Alexa for an immersive search experience.
{'neg': 0.0, 'neu': 0.892, 'pos': 0.108, 'compound': 0.4588}


Love Alexa especially the touch screen - can play games- like Jeopardy
{'neg': 0.161, 'neu': 0.365, 'pos': 0.474, 'compound': 0.7184}


I love the Alexa system of assistance. This version is great but the issue is that people can very easily drop in on video call and you won‚Äôt get to have to the option to pick up. You need to tweak the settings to make sure you keep it private. I installed a screen protector that covers the camera when not in use.
{'neg': 0.044, 'neu': 0.789, 'pos': 0.167, 'compound': 0.8211}


The echo show does so much,the music line up is good,it can do anything.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


This was bought for a gift. But it looks nice. Just what was expect

{'neg': 0.0, 'neu': 0.577, 'pos': 0.423, 'compound': 0.7645}


Awesome all around product - great picture and like the voice control.
{'neg': 0.0, 'neu': 0.428, 'pos': 0.572, 'compound': 0.8934}


I knew this item would be great, but the Echo Show has exceeded my expectations.
{'neg': 0.0, 'neu': 0.836, 'pos': 0.164, 'compound': 0.3716}


Alexa has changed my life. I now own 3 Alexas. O
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


Great product with cool features! Easy to install! I love the way it looks and feels.
{'neg': 0.0, 'neu': 0.439, 'pos': 0.561, 'compound': 0.9335}


I don't know why I waited so long to bring Alexa into my home. It's how I listen to my music now, dictate my shopping and to-do lists and get my morning weather report just to name a few. I highly recommend this item.
{'neg': 0.0, 'neu': 0.932, 'pos': 0.068, 'compound': 0.4201}


The show works great with my other products. Love it
{'neg': 0.0, 'neu': 0.491, 'pos': 0.509, 'compound': 0.8519}


I was he


I use alexa to control my smart home and get internet information
{'neg': 0.0, 'neu': 0.787, 'pos': 0.213, 'compound': 0.4019}


Best Xmas gift and great product made by amazon. Now my fam is all connected.
{'neg': 0.0, 'neu': 0.46, 'pos': 0.54, 'compound': 0.9169}


bought these for family members out of state for Christmas, finding it is way more than expected. Can talk face to face with them, make shopping lists, listen to stories, play interactive trivia music games, is there no end to possibilities? They all LOVED their gift (thank you Best Buy for great sale to make affordable)...
{'neg': 0.031, 'neu': 0.71, 'pos': 0.259, 'compound': 0.9519}


We purchased the Echo Show after using the Dot for a couple of weeks. The first thing I noticed is that the Dot‚Äôs ability to pick up your voice is by far much better than the Show. However, I think the Show‚Äôs speaker is better for listening to music.The screen on the Show is handy for looking at recipes, seeing the shopping list or wat

{'neg': 0.076, 'neu': 0.835, 'pos': 0.09, 'compound': 0.222}


This product ties in to the Amazon ecosystem better than the Dot or Echo. It provides better sound quality and plays video. Pairing this off with Nest cameras has been the best. So quick and easy. I highly recommend this product.
{'neg': 0.0, 'neu': 0.627, 'pos': 0.373, 'compound': 0.9562}


Alexa can do it all! Having a severe hearing loss, the 'Show' gives me visuals along with captions so that I can follow what Alexa is saying!
{'neg': 0.193, 'neu': 0.807, 'pos': 0.0, 'compound': -0.6688}


The Echo show with video screen is a nice upgrade to the other echo‚Äôs. News briefs with videos are nice. Very good sound quality for playing music.
{'neg': 0.0, 'neu': 0.675, 'pos': 0.325, 'compound': 0.8622}


This is the only one you need ! it's like having the Dot, Echo and IHome all in one with a screen !
{'neg': 0.0, 'neu': 0.86, 'pos': 0.14, 'compound': 0.4738}


So great to instantly see family members to chat, demonstrate, c

In [7]:
class node:
    def __init__(self, value):
        self.children = set()
        self.value = value
        
print(parents_counter.most_common(1)[0][0])
root = node(parents_counter.most_common(1)[0][0])

currentNode = root
for tup in newlist:
    #print(tup)
    if tup[0] == currentNode.value:
        currentNode.children.add(node(tup[1]))

for i in root.children:
    currentNode = i
    for tup in newlist:
        #print(tup)
        if tup[0] == currentNode.value:
            currentNode.children.add(node(tup[1]))
    


    

echo show


In [8]:
from anytree import Node, RenderTree

root = Node(parents_counter.most_common(1)[0][0])

currentNode = root

#for all the tuples of parents and children
for tup in newlist:
    #print(tup)
    if tup[0] == currentNode.name: #if we are looking at the node that is the parent in this tuple
        exists = False
        for i in currentNode.children: # look through current node children
            if i.name == tup[1]: #
                exists = True
        if(not exists):
            Node(tup[1], parent=currentNode)


for i in root.children:
    currentNode = i
    for tup in newlist:
        #print(tup)
        if tup[0] == currentNode.name:
            exists = False
            for k in currentNode.children:
                if k.name == tup[1]:
                    exists = True
            if(not exists):
                Node(tup[1], parent=currentNode)
            
                        
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))
    

echo show
├── much, great item
│   └── home
├── 3rd one
│   └── presents
├── great addition
│   ├── echo family
│   ├── connected home
│   ├── smart home
│   ├── sale
│   ├── kitchen
│   ├── amazon family
│   ├── amazon device family
│   ├── home
│   ├── part
│   ├── us
│   ├── alexa products
│   └── echo devices
├── best feature
│   └── us
├── game changer
│   └── echo
├── kitchen
│   ├── good sound
│   ├── hub
│   ├── reference
│   ├── recipes
│   ├── main bathroom
│   ├── music
│   ├── song lyrics
│   ├── everything
│   ├── house
│   ├── sound
│   ├── video display
│   ├── christmas music
│   ├── countless things
│   ├── 5 arlo pro cameras
│   ├── easy review
│   ├── great recipes
│   ├── screen
│   ├── whole family
│   ├── front door
│   ├── ipad
│   ├── holidays
│   ├── places
│   ├── thing
│   ├── kids
│   ├── controls lights
│   ├── receipts
│   ├── better voice recognition
│   ├── echo show
│   ├── numerous alexa devices
│   ├── streaming music
│   ├── calendar
│   ├── amazon m

In [9]:
'''def remove_child_from_path(path):
    tNode = root
    if(root.name == path[0])
        for i in range(1, len(path)):
            for j in tNode.children:
                if j.name == path[i]:
                    tNode = j
                    break
    tNode = tNode.parent'''

'def remove_child_from_path(path):\n    tNode = root\n    if(root.name == path[0])\n        for i in range(1, len(path)):\n            for j in tNode.children:\n                if j.name == path[i]:\n                    tNode = j\n                    break\n    tNode = tNode.parent'

In [10]:
def get_all_paths():
    paths = []
    for i in root.children:
        for j in i.children:
            paths.append((root.name, i.name, j.name))
    return paths
    
all_patharinos = get_all_paths()

In [11]:
def happy_children(path):
    node_pointer = root
    node_pointer.name = path[0]
    for i in range(len(path)-1):
        for child in node_pointer.children:
            if child.name == path[i+1]:
                node_pointer = child
    abc = node_pointer.parent
    our_list = list(abc.children)
    our_list = [i for i in our_list if i!=node_pointer]
    abc.children = tuple(our_list)


In [12]:
hyperparameter = -6
counter = 0
for i in all_patharinos:
    #print(dict_for_freq[ (i[0], i[1]) ])
    #print(dict_for_pos[ (1, i[1]) ])
    print(i)
    print(dict_for_freq[ (i[0], i[1]) ] + dict_for_pos[ (1, i[1]) ] )
    if(dict_for_freq[ (i[0], i[1]) ] + dict_for_pos[ (1, i[1]) ] < hyperparamter):
        counter += 1
        happy_children(i)
        

print(counter)

('echo show', 'much, great item', 'home')
-inf


NameError: name 'hyperparamter' is not defined

In [ ]:
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))